In [121]:
import numpy as np
import pandas as pd
import os
import json
print ('Loading...')

dataRange = 200000
targetId = 1490281

data = pd.read_csv("./data/KaDo_clean.csv")

print ('Data Loaded.'
       'Now creating list...')

maille = data.sort_values(by=['FAMILLE', 'UNIVERS', 'MAILLE'], ascending=[True, True, True])
maille = maille.drop_duplicates(subset=['MAILLE'])
mlen = len(maille)
idList = [i+1 for i in range(mlen)]
maille['PRODUCT_ID'] = idList
maille = maille[['MAILLE','PRODUCT_ID']]

user = data.sort_values(by=['CLI_ID'], ascending=[True])
user = user.drop_duplicates(subset=['CLI_ID'])
newLen = len(user)
userIdList = [i+1 for i in range(newLen)]
user['USER_ID'] = userIdList
user = user[['USER_ID', 'CLI_ID']]

print(userIdList[68196])
# print (user)

Loading...
Data Loaded.Now creating list...
68197


In [122]:

print ('List created.'
       'Now creating Array...')

test = []
tmp_user = []
tmp_merch = []
tmp_merch2 = []
dataLen = len(data)
print(dataLen)

for i in range(dataRange):
       tmp = (maille[maille['MAILLE'] == (data.iloc[i]['MAILLE'])])
       tmp_m = tmp.iloc[0]['PRODUCT_ID']
       tmp2 = (user[user['CLI_ID'] == (data.iloc[i]['CLI_ID'])])
       tmp_u = tmp2.iloc[0]['USER_ID']
       if tmp_u in tmp_user:
              if (tmp_m in tmp_merch[-1]) == False:
                     tmp_merch[-1].append(int(tmp_m))
       else:
              tmp_merch2 = []
              tmp_user.append(tmp_u)
              tmp_merch2.append(tmp_m)
              tmp_merch.append(tmp_merch2)
print(len(tmp_merch))
for i in range(len(tmp_user)):
       test.append([tmp_user[i], tmp_merch[i]])

print (tmp_merch)


List created.Now creating Array...
2661788
34037
[[28, 32], [21, 23], [11], [28], [11, 14], [16, 21, 25, 26, 27, 29], [3], [12, 19, 21], [11, 26], [21], [16], [9, 13], [4, 16, 17, 15, 21, 33], [26, 27, 28], [3], [14, 26, 24, 34], [21], [7, 19], [13, 16, 15, 21, 23, 26, 24, 28, 32], [5, 27], [2, 5, 13], [13, 20, 19, 21, 28], [24], [11, 14, 26, 34], [14, 16, 21], [16, 34], [12, 14, 21, 26], [14, 26, 27, 32], [21], [15, 16, 25], [14, 16, 21], [15, 28, 27], [27], [12, 21, 20, 26, 28, 34], [14, 20, 21, 28, 27], [21], [34], [12, 11, 27, 28, 33, 34], [11, 16, 23, 26, 27, 32], [20], [21], [16], [21], [14], [16], [21], [21], [1, 14, 25, 32, 28], [14, 28], [25], [21, 28], [28], [21], [20, 21], [27], [34], [21], [13, 16, 27], [12, 14, 16, 15], [21], [14], [28], [28], [26], [20], [14, 16], [21, 26], [34], [11, 12, 16, 17, 34], [20], [24], [31], [12], [5, 7, 8, 21, 26, 27], [25], [28], [26], [14, 27, 28], [14], [26, 28], [32], [7, 21, 23], [27], [11, 20, 25], [17, 21, 27], [12, 17, 15, 21, 23, 25, 

In [123]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

In [124]:
import numpy as np

def createSparseMatrice(test):
       newTab = np.zeros([len(test), 34])
       for i in  range(len(test)):
           for j in range(len(test[i])):
                  newTab[i,test[i][j] - 1] = 1
       return newTab

var = createSparseMatrice(tmp_merch)
print(var[0][31])
np.shape(var)

1.0


(34037, 34)

In [125]:

df_sparseMatrix = pd.DataFrame(var, index = [i+1 for i in range(len(tmp_merch))], columns=idList)
df_sparseMatrix

,1,2,3,4,5,...,30,31,32,33,34
1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
34033,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
34034,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
34035,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
34036,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0


In [126]:
kmeans = KMeans(n_clusters=15, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
clusters = kmeans.fit_predict(df_sparseMatrix)

In [127]:
users = np.array([i+1 for i in range(len(tmp_merch))])
clid = np.array(user['CLI_ID'][:len(tmp_merch)])

users_cluster = pd.DataFrame(np.concatenate((users.reshape(-1,1),clid.reshape(-1,1), clusters.reshape(-1,1)), axis = 1), columns = ['userId', 'cli_id', 'Cluster'])

users_cluster.T

,0,1,2,3,4,...,34032,34033,34034,34035,34036
userId,1,2,3,4,5,...,34033,34034,34035,34036,34037
cli_id,1490281,13290776,20163348,20200041,20791601,...,955327915,955328197,955328535,955328939,955329161
Cluster,2,4,12,2,11,...,4,3,4,11,8


In [128]:
def parseData(cli_id):
       tmpuser= user[['CLI_ID']]
       for i in range(len(tmpuser)):
              if (user.iloc[i]['CLI_ID']) == cli_id:
                     return (i)
       return "error"


cid = parseData(targetId)
if cid == "error":
       print(f"Could not find client" )
else:
       tmp = clusters[cid]
       print (tmp)

2


In [129]:
sameClusterUser = []
for i in range(len(clusters)):
       if clusters[i] == tmp:
              sameClusterUser.append(i)
print(sameClusterUser)

[0, 3, 6, 11, 14, 17, 22, 36, 39, 49, 51, 55, 61, 62, 64, 67, 69, 70, 71, 72, 74, 75, 80, 89, 96, 108, 115, 119, 127, 140, 143, 144, 148, 159, 162, 168, 207, 212, 223, 224, 229, 232, 240, 242, 250, 251, 254, 269, 270, 272, 286, 295, 328, 331, 336, 338, 340, 347, 349, 377, 400, 403, 433, 438, 447, 456, 460, 466, 467, 471, 472, 476, 479, 481, 489, 495, 500, 506, 511, 516, 522, 524, 529, 530, 556, 559, 562, 565, 578, 581, 590, 622, 633, 643, 658, 666, 668, 672, 683, 694, 706, 709, 710, 712, 729, 733, 734, 736, 743, 758, 759, 762, 779, 795, 800, 836, 854, 862, 874, 880, 883, 887, 894, 908, 945, 949, 955, 985, 991, 1011, 1016, 1026, 1042, 1048, 1055, 1071, 1104, 1110, 1111, 1115, 1120, 1121, 1122, 1155, 1162, 1175, 1182, 1185, 1190, 1194, 1199, 1208, 1216, 1220, 1246, 1247, 1248, 1251, 1253, 1257, 1258, 1262, 1268, 1276, 1285, 1297, 1298, 1304, 1308, 1311, 1312, 1320, 1334, 1344, 1347, 1351, 1374, 1387, 1419, 1461, 1466, 1468, 1471, 1501, 1502, 1519, 1547, 1594, 1596, 1597, 1599, 1600, 1611

In [130]:
newData = []

for i in range(200000):
       for j in sameClusterUser:
              if data.iloc[i]['CLI_ID'] == user.iloc[j]['CLI_ID']:
                     newData.append(data.iloc[i])


dfNewData = pd.DataFrame(newData)
dfNewData

KeyboardInterrupt: 

In [ ]:
dfNewData.to_csv("./data/maille_Kado.csv")